In [1]:

import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re, collections
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import cohen_kappa_score
import enchant 
import pandas as pd
import spacy
import re
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
import urllib.request

C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://num

In [2]:
dataframe = pd.read_csv('training_set_rel3.tsv', encoding = 'latin-1',sep='\t')
dataframe = dataframe[['essay_id','essay_set','essay','domain1_score']]
dataframe.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


In [3]:
dataframe.set_index('essay_id',inplace=True, drop=True)

In [4]:
dataframe.head()

,essay_set,essay,domain1_score
essay_id,,,
1,1,"Dear local newspaper, I think effects computer...",8
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
5,1,"Dear @LOCATION1, I know having computers has a...",8


In [5]:
import spacy

# load english language model
nlp = spacy.load('en_core_web_sm',disable=['ner','textcat'])

text = "This is a sample sentence."

# create spacy 
doc = nlp(text)

for token in doc:
    print(token.text,'->',token.pos_)

This -> DET
is -> AUX
a -> DET
sample -> NOUN
sentence -> NOUN
. -> PUNCT


In [6]:

def get_number_of_nouns(dataframe):
    feature = []
    for essay in dataframe['essay']:
        noun_count = 0
        doc = nlp(essay)
        for token in doc:
            if token.pos_=='NOUN':
                noun_count=noun_count+1
        feature.append(noun_count)
    return dataframe.assign(num_of_nouns = feature)
dataframe = get_number_of_nouns(dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_nouns
essay_id,,,,
1,1,"Dear local newspaper, I think effects computer...",8,79
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,97
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,72
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,133
5,1,"Dear @LOCATION1, I know having computers has a...",8,106


In [7]:
def get_number_of_verbs(dataframe):
    feature = []
    for essay in dataframe['essay']:
        verb_count = 0
        doc = nlp(essay)
        for token in doc:
            if token.pos_=='VERB':
                verb_count=verb_count+1
        feature.append(verb_count)
    return dataframe.assign(num_of_verbs = feature)
dataframe = get_number_of_verbs(dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_nouns,num_of_verbs
essay_id,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,79,51
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,97,85
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,72,41
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,133,81
5,1,"Dear @LOCATION1, I know having computers has a...",8,106,83


In [8]:
def get_number_of_adpositions(dataframe):
    feature = []
    for essay in dataframe['essay']:
        adposition_count = 0
        doc = nlp(essay)
        for token in doc:
            if token.pos_=='ADP':
                adposition_count=adposition_count+1
        feature.append(adposition_count)
    return dataframe.assign(num_of_adpositions = feature)
dataframe = get_number_of_adpositions(dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_nouns,num_of_verbs,num_of_adpositions
essay_id,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,79,51,45
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,97,85,47
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,72,41,33
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,133,81,56
5,1,"Dear @LOCATION1, I know having computers has a...",8,106,83,37


In [9]:
def get_number_of_adverbs(dataframe):
    feature = []
    for essay in dataframe['essay']:
        adverb_count = 0
        doc = nlp(essay)
        for token in doc:
            if token.pos_=='ADV':
                adverb_count=adverb_count+1
        feature.append(adverb_count)
    return dataframe.assign(num_of_adverbs = feature)
dataframe = get_number_of_adverbs(dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_nouns,num_of_verbs,num_of_adpositions,num_of_adverbs
essay_id,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,79,51,45,18
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,97,85,47,26
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,72,41,33,12
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,133,81,56,26
5,1,"Dear @LOCATION1, I know having computers has a...",8,106,83,37,39


In [10]:
def get_number_of_auxillary(dataframe):
    feature = []
    for essay in dataframe['essay']:
        auxillary_count = 0
        doc = nlp(essay)
        for token in doc:
            if token.pos_=='AUX':
                auxillary_count=auxillary_count+1
        feature.append(auxillary_count)
    return dataframe.assign(num_of_auxillary = feature)
dataframe = get_number_of_auxillary(dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_nouns,num_of_verbs,num_of_adpositions,num_of_adverbs,num_of_auxillary
essay_id,,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,79,51,45,18,23
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,97,85,47,26,14
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,72,41,33,12,18
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,133,81,56,26,31
5,1,"Dear @LOCATION1, I know having computers has a...",8,106,83,37,39,34


In [11]:
def get_number_of_conjunction(dataframe):
    feature = []
    for essay in dataframe['essay']:
        conjunction_count = 0
        doc = nlp(essay)
        for token in doc:
            if token.pos_=='CONJ':
                conjunction_count=conjunction_count+1
        feature.append(conjunction_count)
    return dataframe.assign(num_of_conjunction = feature)
dataframe = get_number_of_conjunction(dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_nouns,num_of_verbs,num_of_adpositions,num_of_adverbs,num_of_auxillary,num_of_conjunction
essay_id,,,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,79,51,45,18,23,0
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,97,85,47,26,14,0
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,72,41,33,12,18,0
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,133,81,56,26,31,0
5,1,"Dear @LOCATION1, I know having computers has a...",8,106,83,37,39,34,0


In [12]:
def get_number_of_c_conjunction(dataframe):
    feature = []
    for essay in dataframe['essay']:
        c_conjunction_count = 0
        doc = nlp(essay)
        for token in doc:
            if token.pos_=='CCONJ':
                c_conjunction_count=c_conjunction_count+1
        feature.append(c_conjunction_count)
    return dataframe.assign(num_of_c_conjunction = feature)
dataframe = get_number_of_c_conjunction(dataframe)
dataframe.head()



,essay_set,essay,domain1_score,num_of_nouns,num_of_verbs,num_of_adpositions,num_of_adverbs,num_of_auxillary,num_of_conjunction,num_of_c_conjunction
essay_id,,,,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,79,51,45,18,23,0,14
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,97,85,47,26,14,0,18
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,72,41,33,12,18,0,16
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,133,81,56,26,31,0,17
5,1,"Dear @LOCATION1, I know having computers has a...",8,106,83,37,39,34,0,16


In [13]:
def get_number_of_determiner(dataframe):
    feature = []
    for essay in dataframe['essay']:
        determiner_count = 0
        doc = nlp(essay)
        for token in doc:
            if token.pos_=='DET':
                determiner_count=determiner_count+1
        feature.append(determiner_count)
    return dataframe.assign(num_of_determiner = feature)
dataframe = get_number_of_determiner(dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_nouns,num_of_verbs,num_of_adpositions,num_of_adverbs,num_of_auxillary,num_of_conjunction,num_of_c_conjunction,num_of_determiner
essay_id,,,,,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,79,51,45,18,23,0,14,32
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,97,85,47,26,14,0,18,45
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,72,41,33,12,18,0,16,30
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,133,81,56,26,31,0,17,54
5,1,"Dear @LOCATION1, I know having computers has a...",8,106,83,37,39,34,0,16,61


In [14]:
def get_number_of_interjection(dataframe):
    feature = []
    for essay in dataframe['essay']:
        interjection_count = 0
        doc = nlp(essay)
        for token in doc:
            if token.pos_=='INTJ':
                interjection_count=interjection_count+1
        feature.append(interjection_count)
    return dataframe.assign(num_of_interjection = feature)
dataframe = get_number_of_interjection(dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_nouns,num_of_verbs,num_of_adpositions,num_of_adverbs,num_of_auxillary,num_of_conjunction,num_of_c_conjunction,num_of_determiner,num_of_interjection
essay_id,,,,,,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,79,51,45,18,23,0,14,32,2
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,97,85,47,26,14,0,18,45,1
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,72,41,33,12,18,0,16,30,0
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,133,81,56,26,31,0,17,54,0
5,1,"Dear @LOCATION1, I know having computers has a...",8,106,83,37,39,34,0,16,61,0


In [15]:
def get_number_of_pronoun(dataframe):
    feature = []
    for essay in dataframe['essay']:
        pronoun_count = 0
        doc = nlp(essay)
        for token in doc:
            if token.pos_=='PRON':
                pronoun_count=pronoun_count+1
        feature.append(pronoun_count)
    return dataframe.assign(num_of_pronoun = feature)
dataframe = get_number_of_pronoun(dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_nouns,num_of_verbs,num_of_adpositions,num_of_adverbs,num_of_auxillary,num_of_conjunction,num_of_c_conjunction,num_of_determiner,num_of_interjection,num_of_pronoun
essay_id,,,,,,,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,79,51,45,18,23,0,14,32,2,36
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,97,85,47,26,14,0,18,45,1,34
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,72,41,33,12,18,0,16,30,0,20
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,133,81,56,26,31,0,17,54,0,20
5,1,"Dear @LOCATION1, I know having computers has a...",8,106,83,37,39,34,0,16,61,0,30


In [16]:
def get_number_of_proper_noun(dataframe):
    feature = []
    for essay in dataframe['essay']:
        proper_noun_count = 0
        doc = nlp(essay)
        for token in doc:
            if token.pos_=='PROPN':
                proper_noun_count=proper_noun_count+1
        feature.append(proper_noun_count)
    return dataframe.assign(num_of_proper_noun = feature)
dataframe = get_number_of_proper_noun(dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_nouns,num_of_verbs,num_of_adpositions,num_of_adverbs,num_of_auxillary,num_of_conjunction,num_of_c_conjunction,num_of_determiner,num_of_interjection,num_of_pronoun,num_of_proper_noun
essay_id,,,,,,,,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,79,51,45,18,23,0,14,32,2,36,5
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,97,85,47,26,14,0,18,45,1,34,3
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,72,41,33,12,18,0,16,30,0,20,1
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,133,81,56,26,31,0,17,54,0,20,30
5,1,"Dear @LOCATION1, I know having computers has a...",8,106,83,37,39,34,0,16,61,0,30,2


In [17]:
def get_number_of_foreign_word(dataframe):
    feature = []
    for essay in dataframe['essay']:
        foreign_word_count = 0
        doc = nlp(essay)
        for token in doc:
            if token.pos_=='FW':
                foreign_word_count=foreign_word_count+1
        feature.append(foreign_word_count)
    return dataframe.assign(num_of_foreign_word = feature)
dataframe = get_number_of_foreign_word(dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_nouns,num_of_verbs,num_of_adpositions,num_of_adverbs,num_of_auxillary,num_of_conjunction,num_of_c_conjunction,num_of_determiner,num_of_interjection,num_of_pronoun,num_of_proper_noun,num_of_foreign_word
essay_id,,,,,,,,,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,79,51,45,18,23,0,14,32,2,36,5,0
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,97,85,47,26,14,0,18,45,1,34,3,0
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,72,41,33,12,18,0,16,30,0,20,1,0
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,133,81,56,26,31,0,17,54,0,20,30,0
5,1,"Dear @LOCATION1, I know having computers has a...",8,106,83,37,39,34,0,16,61,0,30,2,0


In [18]:
def get_number_of_predeterminer(dataframe):
    feature = []
    for essay in dataframe['essay']:
        predeterminer_count = 0
        doc = nlp(essay)
        for token in doc:
            if token.pos_=='PDT':
                predeterminer_count=predeterminer_count+1
        feature.append(predeterminer_count)
    return dataframe.assign(num_of_predeterminer = feature)
dataframe = get_number_of_predeterminer(dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_nouns,num_of_verbs,num_of_adpositions,num_of_adverbs,num_of_auxillary,num_of_conjunction,num_of_c_conjunction,num_of_determiner,num_of_interjection,num_of_pronoun,num_of_proper_noun,num_of_foreign_word,num_of_predeterminer
essay_id,,,,,,,,,,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,79,51,45,18,23,0,14,32,2,36,5,0,0
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,97,85,47,26,14,0,18,45,1,34,3,0,0
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,72,41,33,12,18,0,16,30,0,20,1,0,0
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,133,81,56,26,31,0,17,54,0,20,30,0,0
5,1,"Dear @LOCATION1, I know having computers has a...",8,106,83,37,39,34,0,16,61,0,30,2,0,0


In [19]:
def get_number_of_possesive_ending(dataframe):
    feature = []
    for essay in dataframe['essay']:
        possesive_ending_count = 0
        doc = nlp(essay)
        for token in doc:
            if token.pos_=='POS':
                possesive_ending_count=possesive_ending_count+1
        feature.append(possesive_ending_count)
    return dataframe.assign(num_of_possesive_ending = feature)
dataframe = get_number_of_possesive_ending(dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_nouns,num_of_verbs,num_of_adpositions,num_of_adverbs,num_of_auxillary,num_of_conjunction,num_of_c_conjunction,num_of_determiner,num_of_interjection,num_of_pronoun,num_of_proper_noun,num_of_foreign_word,num_of_predeterminer,num_of_possesive_ending
essay_id,,,,,,,,,,,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,79,51,45,18,23,0,14,32,2,36,5,0,0,0
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,97,85,47,26,14,0,18,45,1,34,3,0,0,0
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,72,41,33,12,18,0,16,30,0,20,1,0,0,0
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,133,81,56,26,31,0,17,54,0,20,30,0,0,0
5,1,"Dear @LOCATION1, I know having computers has a...",8,106,83,37,39,34,0,16,61,0,30,2,0,0,0


In [20]:
def get_number_of_personal_pronoun(dataframe):
    feature = []
    for essay in dataframe['essay']:
        personal_pronoun_count = 0
        doc = nlp(essay)
        for token in doc:
            if token.pos_=='PRP':
                personal_pronoun_count=personal_pronoun_count+1
        feature.append(personal_pronoun_count)
    return dataframe.assign(num_of_personal_pronoun = feature)
dataframe = get_number_of_personal_pronoun(dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_nouns,num_of_verbs,num_of_adpositions,num_of_adverbs,num_of_auxillary,num_of_conjunction,num_of_c_conjunction,num_of_determiner,num_of_interjection,num_of_pronoun,num_of_proper_noun,num_of_foreign_word,num_of_predeterminer,num_of_possesive_ending,num_of_personal_pronoun
essay_id,,,,,,,,,,,,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,79,51,45,18,23,0,14,32,2,36,5,0,0,0,0
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,97,85,47,26,14,0,18,45,1,34,3,0,0,0,0
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,72,41,33,12,18,0,16,30,0,20,1,0,0,0,0
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,133,81,56,26,31,0,17,54,0,20,30,0,0,0,0
5,1,"Dear @LOCATION1, I know having computers has a...",8,106,83,37,39,34,0,16,61,0,30,2,0,0,0,0


In [21]:
def get_number_of_wh_determiner(dataframe):
    feature = []
    for essay in dataframe['essay']:
        wh_determiner_count = 0
        doc = nlp(essay)
        for token in doc:
            if token.pos_=='WDT':
                wh_determiner_count=wh_determiner_count+1
        feature.append(wh_determiner_count)
    return dataframe.assign(num_of_wh_determiner = feature)
dataframe = get_number_of_wh_determiner(dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_nouns,num_of_verbs,num_of_adpositions,num_of_adverbs,num_of_auxillary,num_of_conjunction,num_of_c_conjunction,num_of_determiner,num_of_interjection,num_of_pronoun,num_of_proper_noun,num_of_foreign_word,num_of_predeterminer,num_of_possesive_ending,num_of_personal_pronoun,num_of_wh_determiner
essay_id,,,,,,,,,,,,,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,79,51,45,18,23,0,14,32,2,36,5,0,0,0,0,0
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,97,85,47,26,14,0,18,45,1,34,3,0,0,0,0,0
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,72,41,33,12,18,0,16,30,0,20,1,0,0,0,0,0
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,133,81,56,26,31,0,17,54,0,20,30,0,0,0,0,0
5,1,"Dear @LOCATION1, I know having computers has a...",8,106,83,37,39,34,0,16,61,0,30,2,0,0,0,0,0


In [22]:
def get_number_of_particle(dataframe):
    feature = []
    for essay in dataframe['essay']:
        particle_count = 0
        doc = nlp(essay)
        for token in doc:
            if token.pos_=='PART':
                particle_count=particle_count+1
        feature.append(particle_count)
    return dataframe.assign(num_of_particle = feature)
dataframe = get_number_of_particle(dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_nouns,num_of_verbs,num_of_adpositions,num_of_adverbs,num_of_auxillary,num_of_conjunction,num_of_c_conjunction,num_of_determiner,num_of_interjection,num_of_pronoun,num_of_proper_noun,num_of_foreign_word,num_of_predeterminer,num_of_possesive_ending,num_of_personal_pronoun,num_of_wh_determiner,num_of_particle
essay_id,,,,,,,,,,,,,,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,79,51,45,18,23,0,14,32,2,36,5,0,0,0,0,0,16
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,97,85,47,26,14,0,18,45,1,34,3,0,0,0,0,0,10
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,72,41,33,12,18,0,16,30,0,20,1,0,0,0,0,0,10
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,133,81,56,26,31,0,17,54,0,20,30,0,0,0,0,0,23
5,1,"Dear @LOCATION1, I know having computers has a...",8,106,83,37,39,34,0,16,61,0,30,2,0,0,0,0,0,20


In [23]:
def get_number_of_punctuation(dataframe):
    feature = []
    for essay in dataframe['essay']:
        punctuation_count = 0
        doc = nlp(essay)
        for token in doc:
            if token.pos_=='PUNCT':
                punctuation_count=punctuation_count+1
        feature.append(punctuation_count)
    return dataframe.assign(num_of_punctuation = feature)
dataframe = get_number_of_punctuation(dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_nouns,num_of_verbs,num_of_adpositions,num_of_adverbs,num_of_auxillary,num_of_conjunction,num_of_c_conjunction,...,num_of_interjection,num_of_pronoun,num_of_proper_noun,num_of_foreign_word,num_of_predeterminer,num_of_possesive_ending,num_of_personal_pronoun,num_of_wh_determiner,num_of_particle,num_of_punctuation
essay_id,,,,,,,,,,,,,,,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,79,51,45,18,23,0,14,...,2,36,5,0,0,0,0,0,16,38
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,97,85,47,26,14,0,18,...,1,34,3,0,0,0,0,0,10,34
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,72,41,33,12,18,0,16,...,0,20,1,0,0,0,0,0,10,25
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,133,81,56,26,31,0,17,...,0,20,30,0,0,0,0,0,23,51
5,1,"Dear @LOCATION1, I know having computers has a...",8,106,83,37,39,34,0,16,...,0,30,2,0,0,0,0,0,20,43


In [24]:
def get_number_of_wh_adverb(dataframe):
    feature = []
    for essay in dataframe['essay']:
        wh_adverb_count = 0
        doc = nlp(essay)
        for token in doc:
            if token.pos_=='WRB':
                wh_adverb_count=wh_adverb_count+1
        feature.append(wh_adverb_count)
    return dataframe.assign(num_of_wh_adverb = feature)
dataframe = get_number_of_wh_adverb(dataframe)
dataframe.head()

,essay_set,essay,domain1_score,num_of_nouns,num_of_verbs,num_of_adpositions,num_of_adverbs,num_of_auxillary,num_of_conjunction,num_of_c_conjunction,...,num_of_pronoun,num_of_proper_noun,num_of_foreign_word,num_of_predeterminer,num_of_possesive_ending,num_of_personal_pronoun,num_of_wh_determiner,num_of_particle,num_of_punctuation,num_of_wh_adverb
essay_id,,,,,,,,,,,,,,,,,,,,,
1,1,"Dear local newspaper, I think effects computer...",8,79,51,45,18,23,0,14,...,36,5,0,0,0,0,0,16,38,0
2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,97,85,47,26,14,0,18,...,34,3,0,0,0,0,0,10,34,0
3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,72,41,33,12,18,0,16,...,20,1,0,0,0,0,0,10,25,0
4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,133,81,56,26,31,0,17,...,20,30,0,0,0,0,0,23,51,0
5,1,"Dear @LOCATION1, I know having computers has a...",8,106,83,37,39,34,0,16,...,30,2,0,0,0,0,0,20,43,0


In [26]:
dataframe.to_csv (r'C:\Users\aashish\Desktop\Essay-grading\pos_dataframe2.csv', index = False, header=True)

In [27]:
dataframe.drop(dataframe.index[1783:],0,inplace=True)